### Using tools and creating agents
#### 1- llm just output text, they can't take actions, here comes agents--agents are systems built on llm as reasoning engine(decision making and action taking)
#### 2- AI-AGENT=imitate Human

In [1]:
# in built tools creation 
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun # tools that searches arxiv api
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper # wrapper that use api to conduct search and fetch summaries

In [2]:
# arxiv tool
arxiv_api_wrapper= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

'arxiv'

In [3]:
# wiki tool
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [4]:
# inbuilt tools
tools = [wiki_tool,arxiv_tool]

In [5]:
# creating custom tools[RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://python.langchain.com/docs/how_to/")
docs = loader.load()
splitted_docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents=splitted_docs,embedding=OpenAIEmbeddings())
web_retriever = vectorstore.as_retriever()
web_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002517ECE43D0>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=web_retriever,name="custom web retriver",description="serach anything about how to use____ in langchain")
retriever_tool

Tool(name='custom web retriver', description='serach anything about how to use____ in langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002517B9E7C40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002517ECE43D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002517BD9E340>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002517ECE43D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'

In [8]:
# list of tools 
tools_new = [wiki_tool,arxiv_tool,retriever_tool]
tools_new

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Admin\\Desktop\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='custom web retriver', description='serach anything about how to use____ in langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002517B9E7C40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [9]:
# running tools with agents
import os
from dotenv import load_dotenv
load_dotenv()

import openai
from langchain_groq import ChatGroq

groq_api_key=os.getenv('GROQ_API_KEY')
openai.api_key = os.getenv("OPENAI_API_KEY")

llm_model = ChatGroq(api_key=groq_api_key)


In [12]:
# creating a prompt template
# using chatprompt template we can create a complete prompt for scratch

from langchain import hub # to pull an object from hub which is already predefined
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
# agents
from langchain.agents import create_openai_tools_agent # to create an agent that uses openai tools
agent = create_openai_tools_agent(llm=llm_model,tools=tools_new,prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [14]:
# to execute the agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools_new,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [17]:
agent_executor.invoke({"input":"What is machine learning"})
# here from tools agent itself used wikipedia to answer the question



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a subfield of artificial intelligence that focuses on the development of algorithms capable of learning from data and generalizing to unseen data, allowing tasks to be performed without explicit instructions. tasks without explicit instructionMachine learning is a subfield of artificial intelligence that focuses on the development of algorithms capable of learning from data and generalizing to unseen data, allowing tasks to be performed without explicit instructions. It is a method of data analysis that automates the building of analytical models, enabling systems to find hidden insights without being explicitly programmed where to look. Machine learning is used in a range of computing tasks where designing and programming explicit algorithms is infeasible. Example applications include email filtering, detection of network intrud

{'input': 'What is machine learning',
 'output': 'Machine learning is a subfield of artificial intelligence that focuses on the development of algorithms capable of learning from data and generalizing to unseen data, allowing tasks to be performed without explicit instructions. It is a method of data analysis that automates the building of analytical models, enabling systems to find hidden insights without being explicitly programmed where to look. Machine learning is used in a range of computing tasks where designing and programming explicit algorithms is infeasible. Example applications include email filtering, detection of network intruders, and computer vision.'}

In [21]:
agent_executor.invoke({"input":"How to chain runnables"})
# agent is able to retrive imfo using tools that we custom created



> Entering new AgentExecutor chain...

Invoking: `custom web retriver` with `{'query': 'how to chain runnables in langchain'}`


How to: chain runnables
How to: stream runnables
How to: invoke runnables in parallel
How to: add default invocation args to runnables
How to: turn any function into a runnable
How to: pass through inputs from one chain step to the next
How to: configure runnable behavior at runtime
How to: add message history (memory) to a chain
How to: route between sub-chains
How to: create a dynamic (self-constructing) chain
How to: inspect runnables
How to: add fallbacks to a runnable
How to: pass runtime secrets to a runnable

Components​
These are the core building blocks you can use when building applications.
Prompt templates​
Prompt Templates are responsible for formatting user input into a format that can be passed to a language model.

How to: use few shot examples
How to: use few shot examples in chat models
How to: partially format prompt templates
How to: com

{'input': 'How to chain runnables',
 'output': 'Based on the information provided, it seems like you are looking for guidance on how to chain runnables in LangChain. Here are some steps you can follow:\n\n1. Create the runnables that you want to chain. Runnables are the core building blocks in LangChain, and they can be created using various components such as prompt templates, LangChain Expression Language, and LangGraph.\n2. Determine the order in which you want to chain the runnables. The order of chaining will depend on the specific use case and the desired output.\n3. Pass inputs from one chain step to the next. When chaining runnables, you can pass inputs from one step to the next by specifying the input variable in the next runnable.\n4. Use LangChain Expression Language (LCEL) to create arbitrary custom chains. LCEL is built on the Runnable protocol and provides a way to create complex chains using a simple and expressive language.\n5. Use the LangGraph documentation to build r

In [22]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about"})
# agent is able to retrive info from arxiv
# here in three invokes we didn't mention which tool to use, agent itself decided to use which tool



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks The paper with the identifier "1706.03762" is titled "Attention Is All You Need" and was published on August 2, 2023. The authors of this paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper discusses sequence transduction models that are based on complex recurrent or convolutional neural networks.

> Finished chain.


{'input': "What's the paper 1706.03762 about",
 'output': 'The paper with the identifier "1706.03762" is titled "Attention Is All You Need" and was published on August 2, 2023. The authors of this paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper discusses sequence transduction models that are based on complex recurrent or convolutional neural networks.'}